In [2]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager


In [3]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
with requests.Session() as s:
   city = 'dallas/' #*****change this city to what you want*****
   url = 'https://www.zillow.com/homes/for_sale/'+ city    
   r = s.get(url, headers=req_headers)

In [5]:
soup = bs(r.content, 'html.parser')

In [122]:
listing = soup.find_all('ul', class_='photo-cards photo-cards_wow photo-cards_short photo-cards_extra-attribution')


In [127]:
address_list = []
price_list = []
list_card = []

for lists in listing:
    address = lists.find_all('address', class_='list-card-addr')
    price = lists.find_all('div', class_='list-card-price')
    bd_ba_size = lists.find_all('ul', class_='list-card-details')

for bd in bd_ba_size:
    info = bd.find_all('li', class_='')
    for i in info:
        list_card.append(i.text)

for a in address:
    address_list.append(a.text)

for price in price:
    price_list.append(price.text)

In [128]:
# slice list card to extract beds, baths, size
bds = list_card[0::3]
baths = list_card[1::3]
size = list_card[2::3]

In [130]:
house_info = pd.DataFrame(address_list)
house_info.head()

,0
0,"6220 Bentwood Trl APT 206, Dallas, TX 75252"
1,"2827 Curvilinear Ct, Dallas, TX 75227"
2,"132 W Woodin Blvd, Dallas, TX 75224"
3,"7311 Hardwood Trl, Dallas, TX 75249"
4,"3609 Patience Blvd, Dallas, TX 75236"


In [131]:
house_info['price'] = price_list
house_info['beds'] = bds
house_info['baths'] = baths
house_info['size'] = size

In [132]:
house_info.head()

,0,price,beds,baths,size
0,"6220 Bentwood Trl APT 206, Dallas, TX 75252","$175,000",1 bd,2 ba,807 sqft
1,"2827 Curvilinear Ct, Dallas, TX 75227","$245,000",4 bds,2 ba,"2,200 sqft"
2,"132 W Woodin Blvd, Dallas, TX 75224","$275,000",4 bds,2 ba,"1,800 sqft"
3,"7311 Hardwood Trl, Dallas, TX 75249","$209,900",3 bds,2 ba,"1,902 sqft"
4,"3609 Patience Blvd, Dallas, TX 75236","$324,990",4 bds,2 ba,"2,067 sqft"


In [ ]:
# Clean up: strip 'bds', 'ba', 'sqft', and convert price to only numerical 
